In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
def rho_shock_tube(x, rhoL, rhoR):
    rho = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    rho[mask1] = rhoL
    rho[mask2] = rhoR
    return rho

In [4]:
def u_shock_tube(x, uL, uR):
    u = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    u[mask1] = uL
    u[mask2] = uR
    return u

In [5]:
def P_shock_tube(x, PL, PR):
    P = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    P[mask1] = PL
    P[mask2] = PR
    return P

In [6]:
nx = 81
L = 10.0
dx = 0.25
dt = 0.0002
gamma = 1.4
rhoL = 1.0
rhoR = 0.125
uL = 0.0
uR = 0.0
PL = 100.0
PR = 10.0
nt = 50

x = numpy.linspace(-L, L, num=nx)

rho0 = rho_shock_tube(x, rhoL, rhoR)
u0 = u_shock_tube(x, uL, uR)
P0 = P_shock_tube(x, PL, PR)

In [7]:
def e(P, rho, gamma):
    e = P / ((gamma - 1) * rho)
    return e

In [8]:
U0 = numpy.array([rho0.copy(),
                  u0.copy() * rho0.copy(),
                  P0.copy() * (e(P0, rho0, gamma) * 0.5 * u0.copy()**2)])

In [9]:
def flux(U, gamma):
    F = numpy.zeros_like(U)
    F[0] = U[1]
    F[1] = U[1]**2 / U[0] + (gamma - 1) * (U[2] - 0.5 * U[1]**2 / U[0])
    F[2] = (U[2] + (gamma - 1) * (U[2] - 0.5 * U[1]**2 / U[0])) * U[1] / U[0]
    return F

In [10]:
def rich(U0, nt, gamma, dt, dx, bc_values):
    U = U0.copy()
    U_half = U.copy()
    for n in range(nt):
        F = flux(U, gamma)
        U_half[:,1:-1] = 0.5 * (U[:,2:] + U[:,:-2]) - dt / 2*dx * (F[:,2:] - F[:,:-2])
        F = flux(U_half, gamma)
        U[:,1:-1] = U[:,1:-1] - dt / dx * (F[:,2:] - F[:,:-2])
        U[:,0] = bc_values[0]
        U[:,-1] = bc_values[1]
    return U

In [11]:
U = rich(U0, nt, gamma, dt, dx, (U0[:,0], U0[:,-1]))